# Example 1: Configurations

In [ ]:
data = {'1':{'id':[0,1,2],
             'position':[(0,1,0),(1,1,0),(2,0,1)],
             'name':['A','B','C'],
             'charge':[1,-1,.5],
             'c1':'blue','c2':'red'},
        '2':{'id':[0,1,2],
             'position':[(0,1,0),(.5,0,.5),(1,1,1)],
             'name':['A','B','C'],
             'charge':[1,-1,.5],
             'c1':'blue','c2':'red'}}

In [ ]:
import pandas as pd
import pandas3js as pjs

def change_func(cdata,options):
    df = pd.DataFrame(cdata)[['id','position']]
    ctype = options.get('color','c1')
    df['color'] = cdata[ctype]
    df['radius'] = 0.5
    df['label'] = cdata['name']
    df['other_info'] = ['a sphere<br>charge = %.2f $C.m^{-3}$' % q 
                        for q in cdata['charge']]
    df['otype'] = 'pandas3js.models.Sphere'
    return df

gui, gcollect = pjs.views.create_gui(change_func,data,
                    height=200,width=200,camera_position=[.5,0,-4],
                    view=(3,-3,-3,3),otype_column='otype',
                    opts_dd={'color':['c1','c2']})

In [ ]:
gui

In [ ]:
trait_df = gcollect.trait_df()
trait_df

In [ ]:
trait_df.transparency = 0.1
gcollect.change_by_df(trait_df.loc[0:1],
                      remove_missing=True,
                      otype_column='otype')

# Example 2: Object Types

In [ ]:
import numpy as np 
import pandas as pd
import pandas3js as pjs

def change_func(cdata,options):
    # change id each time to generate new object
    df = pd.DataFrame({
        'id':[np.random.randint(10**3)],
        'color':[options['Color']],
        'transparency':.8,
        'otype':['pandas3js.models.'+options['Object']],
         'position':[(options['x'],0,0)],
         'visible':[options['Visible']]})
    if 'Triclinic' in options['Object']:
        pjs.utils.tuple_to_df(df,'b',(0.5,1,0))
        pjs.utils.tuple_to_df(df,'c',(0,.5,1))
    return df

gui, gcollect = pjs.views.create_gui(change_func,
       opts_dd={'Visible':[True,False],
                'Object':['Sphere','Box','Line', 
                'TriclinicWire','TriclinicSolid',
                'Octahedron','Icosahedron','Circle',
                'Plane']},
       opts_color={'Color':'red'},opts_slide={'x':range(0,5)},
       height=200,width=200,add_labels=False,
       view=(3,-3,-2,4),otype_column='otype')

In [ ]:
gui